In [2]:
%matplotlib inline
import numpy as np
import lmfit
import matplotlib.pyplot as plt
import pandas as pd
import corner
import astropy
import astropy.units as u

ImportError: No module named lmfit

In [ ]:
columns=['name','redshift','distance_mod','DM_err','SN_prob']
rawdat=pd.read_csv('SCPUnion2.1_mu_vs_z.txt', sep='\s',names=columns, header=5, engine='python')

In [ ]:
data=np.sort(rawdat.T, axis=0)
zs = rawdat.redshift
mm = rawdat.distance_mod
dm = rawdat.DM_err

In [ ]:
from astropy.cosmology import LambdaCDM

In [ ]:
def DM(z,H0,Om,Ode):
    cosmo = LambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3, Ode0=0.7)
    DL=cosmo.luminosity_distance(z)
    mag=5*np.log10((DL/(10*u.pc)).decompose())
    return mag

In [ ]:
p = lmfit.Parameters()
p.add_many(('Om',0.3),('H0',70.0),('Ode',0.7))

def residual(p):
    v = p.valuesdict()
    return (DM(zs,v['H0'],v['Om'],v['Ode'])-mm)/dm

In [ ]:
mi = lmfit.minimize(residual, p, method='nelder', nan_policy='omit')

In [ ]:
s = (zs.sort_values()).index
plt.errorbar(zs[s],mm[s],yerr=dm[s],fmt='o',alpha=0.4)
bestOm = mi.params.valuesdict()['Om']
bestH0 = mi.params.valuesdict()['H0']
bestOde =mi.params.valuesdict()['Ode']
plt.plot(zs[s],DM(zs[s],bestH0,bestOm,bestOde))
plt.xlabel('Redshift (z)')
plt.ylabel('Distance Modulus (mag)')
lmfit.report_fit(mi)

In [ ]:
res=lmfit.minimize(residual, method='emcee', nan_policy='omit', burn=600, steps=1500, thin=1, params=mi.params)

In [ ]:
corner.corner(res.flatchain,labels=mi.var_names, truths=list(mi.params.valuesdict().values()))
   